<a href="https://colab.research.google.com/github/kenken0830/preprocessing/blob/master/%E5%89%8D%E5%87%A6%E7%90%86_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter2-2　テーブルデータの前処理

※サポートサイトから2章用のデータをダウンロードし、「sales201907.csv」「sales201907.tsv」をセッションストレージ（P.312）にアップして進めてください。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 必要なライブラリをインポート
import pandas as pd
#csvから読み込む
df = pd.read_csv('sales201907.csv')
#tsvから読み込む
df = pd.read_table('sales201907.tsv')
#tsvをread_csvで読み込む
df = pd.read_csv('sales201907.tsv', sep='\t')

In [ ]:
# !注意！実行するとエラーが出ます。
#エンコードがsjisの場合
df = pd.read_csv('sales201907.csv', encoding='sjis')

# Chapter2-3　データの結合と集約

※サポートサイトから2章用のデータをダウンロードし、「sales201907.csv」「sales201907.tsv」をセッションストレージ（P.312）にアップして進めてください。

## 2-3-1　縦結合

In [ ]:
import pandas as pd
#3ヶ月分読み込む
df_201907 = pd.read_csv('sales201907.csv')
df_201908 = pd.read_csv('sales201908.csv')
df_201909 = pd.read_csv('sales201909.csv')
#縦に結合
df = pd.concat([df_201907, df_201908, df_201909])

## 2-3-2　ID単位で値を集約

#### 2-3-2-1　集約対象が数値である場合

In [ ]:
print(df)

In [ ]:
pd.pivot_table(df, index='顧客ID', columns='商品名', aggfunc='sum')

## 2-3-3　横結合

In [ ]:
#属性データ読み込み
df_zokusei = pd.read_csv('zokusei.csv')
print (df_zokusei)

    顧客ID        生年月日  性別  都道府県  市区町村
0  10001   1982/2/25   1  神奈川県   川崎市
1  10002    1991/8/2   1   埼玉県   朝霞市
2  10003  1976/10/21   1  神奈川県  相模原市
3  10004   1999/1/22   2   千葉県   千葉市


In [ ]:
#ID単位で集約したテーブルに属性を紐づけます
pd.merge(
    df_zokusei,#属性データ
    pd.pivot_table(df, index='顧客ID', columns='商品名', aggfunc='sum').reset_index(),#顧客IDで紐付けるためindexを解除しておく
    on = '顧客ID',
    how = 'left' #左を優先して結合
)

# Chapter2-4　テーブルデータの理解

※サポートサイトから2章用のデータをダウンロードし、「adult.data」をセッションストレージ（P.312）にアップして進めてください。

## 2-4-1　探索的データアナリシス（EDA）

In [ ]:
########################
#サンプルデータ  準備
#calidornia housing
########################
from sklearn.datasets import fetch_california_housing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#欠損をランダムに作成するため乱数オブジェクト作成
rng = np.random.RandomState(0)
#california_housing 読み込む
dataset = fetch_california_housing()
#説明変数を取得
X_full = dataset.data
#被説明変数を取得
y_full = dataset.target
#全てのレコードでは大きいので10サンプルごとに抽出
X_full = X_full[::10]
y_full = y_full[::10]
#欠損作成のためにレコード数とカラム数を取得
n_full_samples, n_full_features = X_full.shape
missing_full_samples = np.arange(n_full_samples)
#意図的にレコードの半分を欠損させる
missing_full_samples = np.random.choice(missing_full_samples,len(missing_full_samples)//2,replace=False)
X_missing = X_full[missing_full_samples].copy()
y = y_full
n_samples, n_features = X_missing.shape
#欠損となる列をランダムに決める
missing_features = rng.choice(n_features, n_samples, replace=True)
#欠損代入
missing_samples = np.arange(n_samples)
X_missing[missing_samples, missing_features] = np.nan
#欠損させない配列
X_not_missing = X_full[np.setdiff1d(np.arange(n_full_samples), missing_full_samples)].copy()
#欠損させる配列とさせない配列を結合
X = np.concatenate([X_not_missing,X_missing])
#データフレーム化
df_california = pd.DataFrame(np.concatenate([X,y.reshape(-1,1)],axis=1))
df_california.columns=dataset.feature_names+['Target']

########################
#サンプルデータ  準備
#Adult Data Set
########################
df_adult = pd.read_csv('adult.data',header=None)
#変数名付与
df_adult.columns = ["Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

## 2-4-2　テーブル全体の理解

In [ ]:
#テーブルのサイズ表示
print (df_california.shape)
#カラムごとの欠損ではないレコード数 とデータ型
print (df_california.info())

In [ ]:
#カラムごと欠損数
print (df_california.isnull().sum())

## 2-4-3　数値変数の分布

In [ ]:
#統計量算出
df_california.describe()

In [ ]:
#変数の箱ひげ図を描く
plt.figure(figsize=(14,10))
for i in np.arange((df_california.shape[1])):
    plt.subplot(3,3,i+1)
    plt.boxplot(df_california.dropna().iloc[:,i])
    plt.title(df_california.columns[i])

In [ ]:
#変数のヒストグラムを描く
plt.figure(figsize=(14,10))
for i in np.arange((df_california.shape[1])):
    plt.subplot(3,3,i+1)
    plt.hist(df_california.dropna().iloc[:,i])
    plt.title(df_california.columns[i])

## 2-4-4　カテゴリカル変数の分布

In [ ]:
#カテゴリカル変数の要約を確認
df_adult.describe(include='O')

In [ ]:
#数値・カテゴリ変数同時に行う
df_adult.describe(include='all').T

In [ ]:
#プロットエリア定義
plt.figure(figsize=(20,20))
#カテゴリカル変数でヒストグラムを書く
for i,j in enumerate(df_adult.describe(include='O').columns):
    plt.subplot(3,3,i+1)
    plt.barh(width=df_adult.loc[:,j].value_counts(),
         y=df_adult.loc[:,j].value_counts().index)
    plt.title(j,fontdict={'fontsize': 20})

## 2-4-5　変数間の相関

In [ ]:
df_california.corr()

In [ ]:
#ヒートマップを作成
import seaborn as sns
plt.figure(figsize=(10,10))
sns.heatmap(df_california.corr(),cmap="YlGnBu",linewidths=5)

# Chapter2-5　カテゴリカル変数の処理

※サポートサイトから2章用のデータをダウンロードし、「adult.data」をセッションストレージ（P.312）にアップして進めてください。

In [ ]:
!pip install category_encoders

## 2-5-1　順序ラベル・エンコーディング

In [ ]:
import pandas as pd
#コーヒーサイズデータフレーム作成
df = pd.DataFrame({
    'SIZE': ['L', 'M', 'S', 'L'],
    'PRICE': ['250', '200', '150', '250']
})

#表示
df

In [ ]:
#category encoder インポート
import category_encoders as ce
#マップを辞書で学習
mapping = [{'col': 'SIZE', 'mapping': {'S': 0, 'M': 1, 'L': 2}}]
#エンコーダーをインスタンス化
enc = ce.OrdinalEncoder(mapping=mapping)
#マップを学習
enc.fit(df)
#変換
enc.transform(df)

## 2-5-2　One-Hotエンコーディング

In [ ]:
#df_adultを読み込む
df_adult = pd.read_csv('adult.data', header=None)
df_adult.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum", "MaritalStatus",
    "Occupation", "Relationship", "Race", "Gender", "CapitalGain",
    "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

#OneHotEncoder 1インスタンス化
#use_cat_names をTrueとすることで変数名が元のカテゴリ名に対応する
encoder = ce.OneHotEncoder(use_cat_names=True)
#学習
encoder.fit(df_adult['MaritalStatus'])
#変換
encoder.transform(df_adult['MaritalStatus'])

#####2-5-2-1 データフレーム全体に適用

In [ ]:
#df全体に適用
encoder = ce.OneHotEncoder(use_cat_names=True)
#学習
encoder.fit(df_adult)
#変換
encoder.transform(df_adult)

####2-5-2-2 ログデータやトランザクションデータの集約への応用

In [ ]:
#購買トランザクション読み込み
df = pd.read_csv('sales201907.csv')
df

In [ ]:
#df全体に適用
encoder = ce.OneHotEncoder(use_cat_names=True)
#学習
encoder.fit(df.drop('購買日時', axis=1))
#変換
df_enc = encoder.transform(df.drop('購買日時', axis=1))

In [ ]:
#pivotで集約する
pd.pivot_table(df_enc, index='顧客ID',aggfunc='sum')

## 2-5-3 ターゲット・エンコーディング

In [ ]:
#データをトレーニング・テストに分ける
from sklearn.model_selection import train_test_split
#NativeCountry取得
X = df_adult['NativeCountry']
#教師データをOneHotEncodingしておく
encoder = ce.OneHotEncoder(use_cat_names=True)
y = encoder.fit_transform(df_adult['Income']).iloc[:, 1]
#データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
#ターゲットエンコーダ インスタンス化
encoder = ce.TargetEncoder()
#トレーニングデータで学習
encoder.fit(X_train,y_train)
#トレーニングを変換
X_train_enc = encoder.transform(X_train)
#テストを変換
X_test_enc = encoder.transform(X_test)

In [ ]:
#元のデータと比較してみる
pd.concat([X_train, X_train_enc], axis=1).drop_duplicates()

# Chapter2-6 欠損値の処理

Chapter 2-5でcategory_encoders をインストールしていない方はインストールしてください。

※サポートサイトから2章用のデータをダウンロードし、「adult.data」をセッションストレージ（P.312）にアップして進めてください。

In [ ]:
# インストールしていない方用 # を外して実行してください
# !pip install category_encoders

## 2-6-2 基本的な欠損処理

In [ ]:
!pip install missingno

In [ ]:
## ここで「You must restart the runtime in order to use newly installed versions.」というメッセージが出たら、「RESTART RUNTIME」ボタンをクリックしてランタイムを再起動してください。
!pip install -U scikit-learn

In [ ]:
#データをトレーニング・テストに分けるモジュール
from sklearn.model_selection import train_test_split
#ロジスティック回帰モデル
from sklearn.linear_model import LogisticRegression
#正答率算出
from sklearn.metrics import accuracy_score
#category encoder インポート
import category_encoders as ce
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#欠損埋めモジュール
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
import missingno as mno

########################
#サンプルデータ  準備
#Adult Data Set
########################
df_adult = pd.read_csv('adult.data', header=None)
#変数名付与
df_adult.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum", "MaritalStatus",
    "Occupation", "Relationship", "Race", "Gender", "CapitalGain",
    "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

#欠損率を20%と設定
p = 0.2
#架空の欠損データ生成
from numpy.random import *
seed(1)
x = pd.Series(np.random.uniform(0.0, 1.0, len(df_adult)))
x.index = df_adult.index

#MCAR作成
df_adult['Age_mcar'] = np.where(x < p, np.nan, df_adult['Age'])

#MAR作成
#全体の欠損率が20%、女性の欠損率が30%となるように設定
p_f = 0.3
p_m = (round(len(df_adult) * p) - df_adult.Gender.value_counts()[1] * p_f
       ) / df_adult.Gender.value_counts()[0]

#男性
ix = df_adult['Gender'] == ' Male'
seed(2)
x = pd.Series(np.random.uniform(0.0, 1.0, len(df_adult)))
x.index = df_adult.index
df_adult['Age_mar'] = np.where((x < p_m) & (ix), np.nan, df_adult['Age'])

#女性
ix = df_adult['Gender'] == ' Male'
seed(3)
x = pd.Series(np.random.uniform(0.0, 1.0, len(df_adult)))
x.index = df_adult.index
df_adult['Age_mar'] = np.where((x < p_f) & (~ix), np.nan, df_adult['Age_mar'])

#MNAR
#40歳以上の一部を欠損とする
ix = df_adult['Age'] >= 40
p_40 = round(len(df_adult) * p) / (ix).sum()
seed(4)
x = pd.Series(np.random.uniform(0.0, 1.0, len(df_adult)))
x.index = df_adult.index
#30%欠損
df_adult['Age_mnar'] = np.where((x < p_40) & (ix), np.nan, df_adult['Age'])

In [ ]:
#ターゲット変数1/0エンコーディング
df_adult['Income'] = (df_adult['Income'] == ' >50K').astype(int)
#特徴量取得
X_adult = df_adult.drop(['Income'], axis=1)
#ターゲット変数を取得
y_adult = df_adult['Income']
#訓練・検証データを分離
X_adult_train, X_adult_test, y_adult_train, y_adult_test = train_test_split(
    X_adult, y_adult, test_size=0.2, random_state=1)

#ターゲットエンコーダ インスタンス化
encoder = ce.TargetEncoder(cols=['Education', 'Occupation', 'NativeCountry'])
enc = encoder.fit(X_adult_train, y_adult_train)
X_adult_train = enc.transform(X_adult_train)
X_adult_test = enc.transform(X_adult_test)

#One-Hotエンコーディング インスタンス化
encoder = ce.OneHotEncoder(
    cols=['WorkClass', 'MaritalStatus', 'Relationship', 'Race', 'Gender'],
    use_cat_names=True)
enc = encoder.fit(X_adult_train)
X_adult_train = enc.transform(X_adult_train)
X_adult_test = enc.transform(X_adult_test)

#One-Hotで冗長になった変数を落とす
drop_l = [
    [col for col in X_adult_train.columns if col.find('WorkClass') >= 0][0], [
        col for col in X_adult_train.columns if col.find('MaritalStatus') >= 0
    ][0], [
        col for col in X_adult_train.columns if col.find('Relationship') >= 0
    ][0], [col for col in X_adult_train.columns if col.find('Race') >= 0][0],
    [col for col in X_adult_train.columns if col.find('Gender') >= 0][0]
]
X_adult_train = X_adult_train.drop(drop_l, axis=1)
X_adult_test = X_adult_test.drop(drop_l, axis=1)

In [ ]:
# 欠損を含むカラムがあるか確認
print(df_adult.isnull().any())

In [ ]:
# カラムごとの欠損の個数を確認
print(df_adult.isnull().sum())

In [ ]:
mno.matrix(df_adult, figsize = (20, 6))

## 2-6-3 欠損を除去する

In [ ]:
#欠損を含む行及び列を落とす (例としてMCARの場合)
#データ作成
X_adult_train_mcar = X_adult_train.drop(['Age','Age_mar','Age_mnar'],axis=1)
print (X_adult_train_mcar.isnull().sum())

#行を落とす
X_adult_train_mcar_dropnar = X_adult_train_mcar.dropna()
print (X_adult_train_mcar_dropnar.isnull().sum())

#列を落とす(axis=1オプション)
X_adult_train_mcar_dropnac = X_adult_train_mcar.dropna(axis=1)
print (X_adult_train_mcar_dropnac.isnull().sum())

## 2-6-4 欠損に値を代入する

### 2-6-4-1 ある定数を代入する

In [ ]:
#元のデータを消さないように
X_adult_train_fillna = X_adult_train.copy()
X_adult_test_fillna = X_adult_test.copy()

#平均値で埋める
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
missing_cols = ['Age_mcar', 'Age_mar', 'Age_mnar']
#訓練データから平均値を算出
imputer.fit(X_adult_train_fillna[missing_cols])
#訓練データ
X_adult_train_fillna[[col + '_mean' for col in missing_cols]] = pd.DataFrame(
    imputer.transform(X_adult_train_fillna[missing_cols]),
    index=X_adult_train.index)
#テストデータ
X_adult_test_fillna[[col + '_mean' for col in missing_cols]] = pd.DataFrame(
    imputer.transform(X_adult_test_fillna[missing_cols]),
    index=X_adult_test.index)

#中央値で埋める
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
missing_cols = ['Age_mcar', 'Age_mar', 'Age_mnar']
imputer.fit(X_adult_train_fillna[missing_cols])
imputer.fit(X_adult_train_fillna[missing_cols])
#訓練データ
X_adult_train_fillna[[col + '_median' for col in missing_cols]] = pd.DataFrame(
    imputer.transform(X_adult_train_fillna[missing_cols]),
    index=X_adult_train.index)
#テストデータ
X_adult_test_fillna[[col + '_median' for col in missing_cols]] = pd.DataFrame(
    imputer.transform(X_adult_test_fillna[missing_cols]),
    index=X_adult_test.index)

#最頻値で埋める
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
missing_cols = ['Age_mcar', 'Age_mar', 'Age_mnar']
imputer.fit(X_adult_train_fillna[missing_cols])
#訓練データ
X_adult_train_fillna[[col + '_mode' for col in missing_cols]] = pd.DataFrame(
    imputer.transform(X_adult_train_fillna[missing_cols]),
    index=X_adult_train.index)
#テストデータ
X_adult_test_fillna[[col + '_mode' for col in missing_cols]] = pd.DataFrame(
    imputer.transform(X_adult_test_fillna[missing_cols]),
    index=X_adult_test.index)

In [ ]:
#MARの代入前後の分布
plt.figure(figsize=(14, 7))
for i, col in enumerate(
    ['Age_mar', 'Age_mar_mean', 'Age_mar_median', 'Age_mar_mode']):
    plt.subplot(2, 2, i + 1)
    plt.hist(
        X_adult_train_fillna['Age'],
        alpha=0.7,
        label='Complete Data',
        bins=range(0, 100, 10))
    plt.legend()
    plt.hist(X_adult_train_fillna[col], alpha=0.7, bins=range(0, 100, 10))
    plt.grid()
    plt.title(col)
    plt.xticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

### 2-6-4-2 他の変数から推定する

In [ ]:
#sklearnのLinearRegressionモジュール読み込む
from sklearn.linear_model import LinearRegression
#線形回帰のインスタンス化
reg1 = LinearRegression()
reg2 = LinearRegression()
reg3 = LinearRegression()

#MCARに対して処理を実行
#使用しない特徴量のリスト特にターゲットエンコーディングした特徴量は除く
ix = X_adult_train['Age_mcar'].isnull()
drop = ['Education', 'Occupation', 'NativeCountry'
        ] + ['Age', 'Age_mcar', 'Age_mar', 'Age_mnar', 'fnlwgt']
x1 = X_adult_train[~ix].drop(drop, axis=1)
x2 = X_adult_train.loc[x1.index, 'Age_mcar']

#線形回帰
reg1.fit(x1, x2)

#訓練データ予測する
X_adult_train_fillna['Age_mcar_reg'] = pd.Series(
    reg1.predict(X_adult_train[ix].drop(drop, axis=1)),
    X_adult_train[ix].index)
X_adult_train_fillna['Age_mcar_reg'] = X_adult_train_fillna[
    'Age_mcar_reg'].fillna(X_adult_train_fillna['Age'])
#検証データ予測する
ixt = X_adult_test['Age_mcar'].isnull()
X_adult_test_fillna['Age_mcar_reg'] = pd.Series(
    reg1.predict(X_adult_test[ixt].drop(drop, axis=1)),
    X_adult_test[ixt].index)
X_adult_test_fillna['Age_mcar_reg'] = X_adult_test_fillna[
    'Age_mcar_reg'].fillna(X_adult_test_fillna['Age'])

In [ ]:
#MARに対して処理を実行
#使用しない特徴量のリスト特にターゲットエンコーディングした特徴量は除く
ix = X_adult_train['Age_mar'].isnull()
drop = ['Education', 'Occupation', 'NativeCountry'
        ] + ['Age', 'Age_mcar', 'Age_mar', 'Age_mnar', 'fnlwgt']
x1 = X_adult_train[~ix].drop(drop, axis=1)
x2 = X_adult_train.loc[x1.index, 'Age_mar']

#線形回帰
reg2.fit(x1, x2)

#訓練データ予測する
X_adult_train_fillna['Age_mar_reg'] = pd.Series(
    reg2.predict(X_adult_train[ix].drop(drop, axis=1)),
    X_adult_train[ix].index)
X_adult_train_fillna['Age_mar_reg'] = X_adult_train_fillna[
    'Age_mar_reg'].fillna(X_adult_train_fillna['Age'])
#検証データ予測する
ixt = X_adult_test['Age_mar'].isnull()
X_adult_test_fillna['Age_mar_reg'] = pd.Series(
    reg1.predict(X_adult_test[ixt].drop(drop, axis=1)),
    X_adult_test[ixt].index)
X_adult_test_fillna['Age_mar_reg'] = X_adult_test_fillna['Age_mar_reg'].fillna(
    X_adult_test_fillna['Age'])

#MNARに対して処理を実行
#使用しない特徴量のリスト特にターゲットエンコーディングした特徴量は除く
ix = X_adult_train['Age_mnar'].isnull()
drop = ['Education', 'Occupation', 'NativeCountry'
        ] + ['Age', 'Age_mcar', 'Age_mar', 'Age_mnar', 'fnlwgt']
x1 = X_adult_train[~ix].drop(drop, axis=1)
x2 = X_adult_train.loc[x1.index, 'Age_mnar']
#線形回帰
reg3.fit(x1, x2)

#訓練データ予測する
X_adult_train_fillna['Age_mnar_reg'] = pd.Series(
    reg3.predict(X_adult_train[ix].drop(drop, axis=1)),
    X_adult_train[ix].index)
X_adult_train_fillna['Age_mnar_reg'] = X_adult_train_fillna[
    'Age_mnar_reg'].fillna(X_adult_train_fillna['Age'])
#検証データ予測する
ixt = X_adult_test['Age_mnar'].isnull()
X_adult_test_fillna['Age_mnar_reg'] = pd.Series(
    reg1.predict(X_adult_test[ixt].drop(drop, axis=1)),
    X_adult_test[ixt].index)
X_adult_test_fillna['Age_mnar_reg'] = X_adult_test_fillna[
    'Age_mnar_reg'].fillna(X_adult_test_fillna['Age'])

In [ ]:
##################MCAR 埋める##################
#使用しない特徴量のリスト特にターゲットエンコーディングした特徴量は除く
drop = ['Education', 'Occupation', 'NativeCountry'
        ] + ['Age', 'Age_mar', 'Age_mnar', 'fnlwgt']

#KNNImputerインスタンス化
imputer = KNNImputer(n_neighbors=2)
#訓練
imputer.fit(X_adult_train.drop(drop, axis=1))
#訓練データに代入
X_adult_train_fillna['Age_mcar_knn'] = pd.DataFrame(
    imputer.transform(X_adult_train.drop(drop, axis=1)),
    index=X_adult_train.drop(drop, axis=1).index,
    columns=X_adult_train.drop(drop, axis=1).columns)['Age_mcar']

#検証データに代入
X_adult_test_fillna['Age_mcar_knn'] = pd.DataFrame(
    imputer.transform(X_adult_test.drop(drop, axis=1)),
    index=X_adult_test.drop(drop, axis=1).index,
    columns=X_adult_test.drop(drop, axis=1).columns)['Age_mcar']

##################MAR 埋める##################
#使用しない特徴量のリスト特にターゲットエンコーディングした特徴量は除く
drop = ['Education', 'Occupation', 'NativeCountry'
        ] + ['Age', 'Age_mcar', 'Age_mnar', 'fnlwgt']

#KNNImputerインスタンス化
imputer = KNNImputer(n_neighbors=2)
#訓練
imputer.fit(X_adult_train.drop(drop, axis=1))
#訓練データに代入
X_adult_train_fillna['Age_mar_knn'] = pd.DataFrame(
    imputer.transform(X_adult_train.drop(drop, axis=1)),
    index=X_adult_train.drop(drop, axis=1).index,
    columns=X_adult_train.drop(drop, axis=1).columns)['Age_mar']

#検証データに代入
X_adult_test_fillna['Age_mar_knn'] = pd.DataFrame(
    imputer.transform(X_adult_test.drop(drop, axis=1)),
    index=X_adult_test.drop(drop, axis=1).index,
    columns=X_adult_test.drop(drop, axis=1).columns)['Age_mar']

 ##################MNAR 埋める##################
#使用しない特徴量のリスト特にターゲットエンコーディングした特徴量は除く
drop = ['Education', 'Occupation', 'NativeCountry'
        ] + ['Age', 'Age_mcar', 'Age_mar', 'fnlwgt']

#KNNImputerインスタンス化
imputer = KNNImputer(n_neighbors=2)
#訓練
imputer.fit(X_adult_train.drop(drop, axis=1))
#訓練データに代入
X_adult_train_fillna['Age_mnar_knn'] = pd.DataFrame(
    imputer.transform(X_adult_train.drop(drop, axis=1)),
    index=X_adult_train.drop(drop, axis=1).index,
    columns=X_adult_train.drop(drop, axis=1).columns)['Age_mnar']

#検証データに代入
X_adult_test_fillna['Age_mnar_knn'] = pd.DataFrame(
    imputer.transform(X_adult_test.drop(drop, axis=1)),
    index=X_adult_test.drop(drop, axis=1).index,
    columns=X_adult_test.drop(drop, axis=1).columns)['Age_mnar']

In [ ]:
#MARの代入前後の分布
plt.figure(figsize=(14, 7))
for i, col in enumerate(['Age_mar', 'Age_mar_reg', 'Age_mar_knn']):
    plt.subplot(2, 2, i + 1)
    plt.hist(
        X_adult_train_fillna['Age'],
        alpha=0.7,
        label='Complete Data',
        bins=range(0, 100, 10))
    plt.legend()
    plt.hist(X_adult_train_fillna[col], alpha=0.7, bins=range(0, 100, 10))
    plt.grid()
    plt.title(col)
    plt.xticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

### 2-6-4-3 多重代入法


In [ ]:
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

In [ ]:
##################MCAR 埋める##################
#使用しない特徴量のリスト特にターゲットエンコーディングした特徴量は除く
drop = ['Education', 'Occupation', 'NativeCountry'
        ] + ['Age', 'Age_mar', 'Age_mnar', 'fnlwgt']

#IterativeImputerインスタンス化
imputer = IterativeImputer(max_iter=5, sample_posterior=True, random_state=123)

#訓練
imputer.fit(X_adult_train.drop(drop, axis=1))
#訓練データに代入
X_adult_train_fillna['Age_mcar_itr'] = pd.DataFrame(
    imputer.transform(X_adult_train.drop(drop, axis=1)),
    index=X_adult_train.drop(drop, axis=1).index,
    columns=X_adult_train.drop(drop, axis=1).columns)['Age_mcar']

#検証データに代入
X_adult_test_fillna['Age_mcar_itr'] = pd.DataFrame(
    imputer.transform(X_adult_test.drop(drop, axis=1)),
    index=X_adult_test.drop(drop, axis=1).index,
    columns=X_adult_test.drop(drop, axis=1).columns)['Age_mcar']

##################MAR 埋める##################
#使用しない特徴量のリスト特にターゲットエンコーディングした特徴量は除く
drop = ['Education', 'Occupation', 'NativeCountry'
        ] + ['Age', 'Age_mcar', 'Age_mnar', 'fnlwgt']

#IterativeImputerインスタンス化
imputer = IterativeImputer(max_iter=5, sample_posterior=True, random_state=123)
#訓練
imputer.fit(X_adult_train.drop(drop, axis=1))
#訓練データに代入
X_adult_train_fillna['Age_mar_itr'] = pd.DataFrame(
    imputer.transform(X_adult_train.drop(drop, axis=1)),
    index=X_adult_train.drop(drop, axis=1).index,
    columns=X_adult_train.drop(drop, axis=1).columns)['Age_mar']

#検証データに代入
X_adult_test_fillna['Age_mar_itr'] = pd.DataFrame(
    imputer.transform(X_adult_test.drop(drop, axis=1)),
    index=X_adult_test.drop(drop, axis=1).index,
    columns=X_adult_test.drop(drop, axis=1).columns)['Age_mar']

##################MNAR 埋める##################
#使用しない特徴量のリスト特にターゲットエンコーディングした特徴量は除く
drop = ['Education', 'Occupation', 'NativeCountry'
        ] + ['Age', 'Age_mcar', 'Age_mar', 'fnlwgt']

#IterativeImputerインスタンス化
imputer = IterativeImputer(max_iter=5, sample_posterior=True, random_state=123)
#訓練
imputer.fit(X_adult_train.drop(drop, axis=1))

#訓練データに代入
X_adult_train_fillna['Age_mnar_itr'] = pd.DataFrame(
    imputer.transform(X_adult_train.drop(drop, axis=1)),
    index=X_adult_train.drop(drop, axis=1).index,
    columns=X_adult_train.drop(drop, axis=1).columns)['Age_mnar']

#検証データに代入
X_adult_test_fillna['Age_mnar_itr'] = pd.DataFrame(
    imputer.transform(X_adult_test.drop(drop, axis=1)),
    index=X_adult_test.drop(drop, axis=1).index,
    columns=X_adult_test.drop(drop, axis=1).columns)['Age_mnar']

In [ ]:
#MARの代入前後の分布
plt.figure(figsize=(14, 7))
for i, col in enumerate(['Age_mar', 'Age_mar_itr']):
    plt.subplot(2, 2, i + 1)
    plt.hist(
        X_adult_train_fillna['Age'],
        alpha=0.7,
        label='Complete Data',
        bins=range(0, 100, 10))
    plt.legend()
    plt.hist(X_adult_train_fillna[col], alpha=0.7, bins=range(0, 100, 10))
    plt.grid()
    plt.title(col)
    plt.xticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

#### 2-6-4-4 数値代入法の選び方

In [ ]:
#MCARの代入法ごとのモデル精度比較
#精度を格納するリスト
l_tr = []
l_te = []
for age in [
        'Age'
] + [col for col in X_adult_train_fillna.columns if col.find('Age_mar_') > -1]:
    #予測に必要な特徴量 'fnlwgt'は重み変数のため除外
    var = [
        col for col in X_adult_train.columns
        if col.find('Age') == -1 and col.find('fnlwgt') == -1
    ] + [age]
    print(age, var)
    #ロジスティック回帰インスタンス化
    reg = LogisticRegression(solver='newton-cg')
    #モデル学習
    reg.fit(X_adult_train_fillna[var], y_adult_train)
    #訓練データでの精度検証
    l_tr.append(
        accuracy_score(y_adult_train, reg.predict(X_adult_train_fillna[var])))
    #検証データでの精度検証
    l_te.append(
        accuracy_score(y_adult_test, reg.predict(X_adult_test_fillna[var])))
    
plt.figure(figsize=(14, 7))
plt.plot(
    ['Age'] +
    [col for col in X_adult_train_fillna.columns if col.find('Age_mar_') >-1],
    l_tr,
    label='Train')
plt.plot(l_te, label='Test')
plt.legend()
plt.title('Impute MCAR')
plt.ylim(0.845,0.855)

## 2-6-5 欠損カテゴリを新たに作成する

In [ ]:
#架空の欠損データ生成
import pandas as pd
import numpy as np
from numpy.random import *
seed(100)
df_adult = pd.read_csv('adult.data', header=None)
#変数名付与
df_adult.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum", "MaritalStatus",
    "Occupation", "Relationship", "Race", "Gender", "CapitalGain",
    "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

#Raceで欠損を作成する
#全体の欠損率が20%、女性の欠損率が30%となるように設定
p = 0.2
p_f = 0.3
p_m = (round(len(df_adult) * p) - df_adult.Gender.value_counts()[1] * p_f
       ) / df_adult.Gender.value_counts()[0]

#男性
ix = df_adult['Gender'] == ' Male'
seed(200)
x = pd.Series(np.random.uniform(0.0, 1.0, len(df_adult)))
x.index = df_adult.index
df_adult['Race_mar'] = np.where((x < p_m) & (ix), np.nan, df_adult['Race'])

#女性
ix = df_adult['Gender'] == ' Male'
seed(300)
x = pd.Series(np.random.uniform(0.0, 1.0, len(df_adult)))
x.index = df_adult.index
df_adult['Race_mar'] = np.where((x < p_f) & (~ix), np.nan,
                                df_adult['Race_mar'])

#欠損を適当なカテゴリ名（"Missing")という文字列で埋める
df_adult['Race_mar'].fillna('Missing', inplace=True)
#集計
print(df_adult['Race_mar'].value_counts())

In [ ]:
#One-Hotエンコーディング インスタンス化
encoder = ce.OneHotEncoder(cols=['Race_mar'], use_cat_names=True)
enc = encoder.fit(df_adult)
df_adult = enc.transform(df_adult)

# Chapter2-7 データスケーリング

※サポートサイトから2章用のデータをダウンロードし、「UCI_Credit_Card.csv」をセッションストレージ（P.312）にアップして進めてください。

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
#preprosess モジュールのインポート
from sklearn import preprocessing
#データ分割用
from sklearn.model_selection import train_test_split
#ロジスティック回帰モデル作成のため
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#可視化のため
import seaborn as sns
import matplotlib.pyplot as plt
#データ読み込み
df = pd.read_csv('UCI_Credit_Card.csv')
#要約確認
df.describe().T

In [ ]:
#train test データ分離
X = df.drop(
    [
        'ID', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE',
        'default.payment.next.month'
    ],
    axis=1)
y = df['default.payment.next.month']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

## 2-7-1 Min-Max法

In [ ]:
#MinMaxScalerのインスタンス化
min_max_scaler = preprocessing.MinMaxScaler()
# train
X_train_min_max = min_max_scaler.fit_transform(X_train)
# test trainを基準にしないとリークしてしまう
X_test_min_max = min_max_scaler.transform(X_test)

#分布描画
fontdic = {'size': 20}
plt.figure(figsize=(14, 7))
#変換前
plt.subplot(1, 2, 1)
plt.hist(X_train['BILL_AMT1'])
plt.title('Before', fontdict=fontdic)
plt.grid()
#変換後
plt.subplot(1, 2, 2)
plt.hist(X_train_min_max[:, 8], color='red')
plt.title('After', fontdict=fontdic)
plt.grid()

## 2-7-2 Zスコア標準化

In [ ]:
#zスコア変換
standard_scaler = preprocessing.StandardScaler()  
# train
X_train_standard = standard_scaler.fit_transform(X_train)
# test trainを基準にしないとリークしてしまう
X_test_standard = standard_scaler.transform(X_test)

#分布描画
fontdic = {'size': 20}
plt.figure(figsize=(14, 7))
#変換前
plt.subplot(1, 2, 1)
plt.hist(X_train['BILL_AMT1'])
plt.title('Before', fontdict=fontdic)
plt.grid()
#変換後
plt.subplot(1, 2, 2)
plt.hist(X_train_standard[:, 8], color='red')
plt.title('After', fontdict=fontdic)
plt.grid()

## 2-7-3 10進スケールの正規化

In [ ]:
#10進変換
#変換用の関数作成
def decimal_fit_trans(x):
    #絶対値の最大値をカラムごとに取得
    temp = np.nanmax(np.abs(x.values),axis=0)
    arr = np.array([])
    #カラムごとに除する定数を取得
    for t in temp:
        i = 0
        while t/10**(i) > 1:
            i += 1
        arr = np.r_[arr,1/10**(i)]
    return  (arr)

In [ ]:
#train
X_train_decimal = X_train*decimal_fit_trans(X_train)
#test  
X_test_decimal = X_test*decimal_fit_trans(X_train)
#分布描画
fontdic = {'size':20}
plt.figure(figsize=(14,7))
#変換前
plt.subplot(1,2,1)
plt.hist(X_train['BILL_AMT1'])
plt.title('Before',fontdict=fontdic)
plt.grid()
#変換後
plt.subplot(1,2,2)
plt.hist(X_train_decimal['BILL_AMT1'],color='red')
plt.title('After',fontdict=fontdic)
plt.grid()

## 2-7-4 スケーリング手法比較

In [ ]:
#変換後比較
plt.figure(figsize=(14, 10))
for i, col in enumerate(X_train.columns):
    plt.subplot(5, 4, i + 1)
    plt.hist(
        pd.DataFrame(X_train_min_max, columns=X_train.columns)[col],
        color='blue',
        alpha=0.5,
        label='Min-Max')
    plt.hist(
        pd.DataFrame(X_train_standard, columns=X_train.columns)[col],
        color='green',
        alpha=0.5,
        label='Standard')
    plt.hist(X_train_decimal[col], color='red', alpha=0.5, label='Decimal')
    plt.title(col)
    plt.legend()

In [ ]:
#モデルの生成
clf = LogisticRegression()
# 精度を格納するリスト
l = []
# 学習
#スケーリングなし
clf.fit(X_train, y_train)
l.append(accuracy_score(y_test, clf.predict(X_test)))
#min_max変換
clf.fit(X_train_min_max, y_train)
l.append(accuracy_score(y_test, clf.predict(X_test_min_max)))
#zスコア変換
clf.fit(X_train_standard, y_train)
l.append(accuracy_score(y_test, clf.predict(X_test_standard)))
#10進変換
clf.fit(X_train_decimal, y_train)
l.append(accuracy_score(y_test, clf.predict(X_test_decimal)))
# 精度一覧
print(l)

# Chapter2-8 データ変換

## 2-8-2 二次型変換

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#人工データを作成
from sklearn.datasets import make_circles
X, y = make_circles(noise=0.02, random_state=0)  #ラベル付き円を作成
plt.figure(figsize=(5, 5))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='winter')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#x軸y軸を説明変数としてロジスティック回帰を適用
clf = LogisticRegression()
X_train = X
y_train = y
clf.fit(X_train, y_train)
accuracy_score(y_train, clf.predict(X_train))

In [ ]:
#2次変換
#モデルの生成
clf_q = LogisticRegression()
Z = (X[:, 0]**(2) + X[:, 1]**(2)).reshape(-1, 1)
X_train = Z
y_train = y
clf_q.fit(X_train, y_train)
accuracy_score(y_train, clf_q.predict(X_train))

In [ ]:
plt.figure(figsize=(5,5))
#通常の方法
x_=np.linspace(-1,1,100)
plt.scatter(X[:,0],X[:,1],c=y,cmap='winter')
#log(p/(1-p)=1/2となる線を描画する
plt.plot(x_,(-1*(clf.coef_[:,0]*x_)-clf.intercept_)/clf.coef_[:,1])
plt.ylim([-1,1])

#2次変換
theta = np.arange(0,2*np.pi+1,0.1)
xlist=[];ylist=[]
r =  ((clf_q.intercept_/(-1*clf_q.coef_))**(1/2))[0,0] #半径
for i in theta:
	xlist.append(r*np.cos(i))
	ylist.append(r*np.sin(i))
plt.plot(xlist,ylist)

## 2-8-3 変換の非多項式近似

In [ ]:
#三角形の合同問題
from pandas import DataFrame
#例として合同な三角形を考えるため描画ライブラリを読み込む

#乱数設定
np.random.seed(1)
ran_l1 = list(np.random.rand(10) * 5)
np.random.seed(2)
ran_l2 = list(np.random.rand(10))

#三角形の座標データを作成
base1 = np.array([0, 1, -1, 0, 1, 0])
base2 = np.array([0, 0, 1, 0, 0, np.sqrt(3)])
data = np.vstack([base1 - i
                  for i in ran_l1[0:5]] + [base2 - i for i in ran_l1[5:]])
l = []
for i, j in zip(data, ran_l2):
    theta = (2 * j) * np.pi
    l.append(
        np.dot(
            np.array([[np.cos(theta), -np.sin(theta)],
                      [np.sin(theta), np.cos(theta)]]),
            i.reshape(3, 2).T).T.reshape(-1))

data = np.vstack(l)

df_data = DataFrame(data, columns=['x1', 'y1', 'x2', 'y2', 'x3', 'y3'])

In [ ]:
print (df_data)

In [ ]:
#描画する
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)

for k, i in enumerate(data):
  print(i, k)
  if k < 5:
      tri = plt.Polygon(
          ((i[0], i[1]), (i[2], i[3]), (i[4], i[5])), fc="blue")
      ax.add_patch(tri)
  else:
      tri = plt.Polygon(((i[0], i[1]), (i[2], i[3]), (i[4], i[5])), fc="red")
      ax.add_patch(tri)
plt.xlim(-7, 7)
plt.ylim(-7, 7)
plt.show()

## 2-8-4 ランク変換

In [ ]:
#人工的にデータを作成
#y ~ N(3x+1,0.2) に従う(x in [0,1])
np.random.seed(seed=1)
x = np.random.rand(100)
y = np.random.normal(3 * x + 1, 0.2)
#xの外れ値を作成
x = np.where(x == max(x), 3, x)
plt.scatter(x=x, y=y)
plt.grid()

In [ ]:
#ランク算出のためのライブラリインポート
from scipy.stats import rankdata
#正規分布の累積分布関数の逆関数を計算するためにインポート
from scipy.stats import norm
#ランク変換関数定義
def rank_fit_transform(x):
  return (norm.ppf((rankdata(x) - (3 / 8)) / (len(x) + (1 / 4))))

In [ ]:
#線形回帰モデルの準備
from sklearn import linear_model

#ランク変換前
clf = linear_model.LinearRegression()
X = x.reshape(-1, 1)
y = y
clf.fit(X, y)
clf.score(X, y)

#ランク変換後
clf_r = linear_model.LinearRegression()
X_r = rank_fit_transform(x).reshape(-1, 1)
y = y
clf_r.fit(X_r, y)
clf_r.score(X_r, y)
#決定係数
print("変換前_{:.5g}".format(clf.score(X, y)), "変換後_{:.5g}".format(
	clf_r.score(X_r, y)))

In [ ]:
#直線当てはめを描画
plt.figure(figsize=(14, 7))
#変換なし
plt.subplot(1,2,1)
plt.scatter(X, y)
plt.scatter(X, clf.predict(X), s=5)
plt.grid()
plt.title('Before')
#ランク変換後
plt.subplot(1,2,2)
plt.scatter(X_r, y)
plt.scatter(X_r, clf_r.predict(X_r), s=5)
plt.grid()
plt.title('After')

## 2-8-5 Box-Cox変換

In [ ]:
#必要なライブラリ読み込む
from scipy import stats  #boxcox変換のため
from sklearn import linear_model  #線形回帰モデル
from sklearn.model_selection import train_test_split  #訓練データ分割
#UCI よりレンタル自転車のデータを読み込む
df = pd.read_csv('hour.csv')
#描画準備
plt.figure(figsize=(14, 7))
#cntの変換前ヒストグラム
plt.subplot(1, 2, 1)
plt.hist(df['cnt'])
plt.title('Before')
plt.grid()
#box-cox変換後の分布
plt.subplot(1, 2, 2)
bc, _ = stats.boxcox(df['cnt'])
plt.hist(bc)
plt.title('After')
plt.grid()

In [ ]:
#データを訓練データとテストデータに分離
X = df['temp'].values
y = df['cnt'].values
X_train, X_test, y_train, y_test = train_test_split(
	X, y, test_size=0.2, random_state=0)

#変換
y_train_bc, lambd = stats.boxcox(y_train)

#変換前回帰
clf = linear_model.LinearRegression()
clf.fit(X_train.reshape(-1, 1), y_train.reshape(-1, 1))

#変換後回帰
clf_bc = linear_model.LinearRegression()
clf_bc.fit(X_train.reshape(-1, 1), y_train_bc.reshape(-1, 1))

In [ ]:
#残差プロット
#比較用y=0の線
x = np.arange(0, 20)
y = 0 * x

plt.figure(figsize=(14, 7))
#変換前
plt.subplot(1,2,1)
plt.scatter(
    clf_bc.predict(X_train.reshape(-1, 1)),
    y_train.reshape(-1, 1) - clf.predict(X_train.reshape(-1, 1)))
plt.plot(x, y, color='red')
plt.title('Before')
#変換後
plt.subplot(1,2,2)
plt.scatter(
    clf_bc.predict(X_train.reshape(-1, 1)),
    y_train_bc.reshape(-1, 1) - clf_bc.predict(X_train.reshape(-1, 1)))
plt.plot(x, y, color='red')
plt.title('After')

# Chapter2-9 次元削減法

In [ ]:
# 日本語フォントの設定★
%%bash
apt-get install -y fonts-ipafont-gothic > /dev/null
cachedir=$(python -c 'import matplotlib as m; print(m.get_cachedir())')
rm -f $cachedir/fontlist-v300.json
pip install japanize-matplotlib > /dev/null

In [ ]:
import matplotlib
from matplotlib import pyplot
import japanize_matplotlib
matplotlib.rc('font', family="IPAexGothic")

###  2-9-1 次元の呪い

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#球面集中問題
def calc_volume(e, D):
	return 1 - (1 - e)**D

vol_list = []
e = 0.01
for i in range(1, 300):
	vol_list.append(calc_volume(e, i))

#図示する
plt.plot(vol_list)
plt.xlabel('次元')
plt.ylabel('球の体積比')
plt.grid()

## 2-9-2 主成分分析（PCA）

In [ ]:
from sklearn.decomposition import PCA  #主成分分析
#ESTAT 県民総生産 (2015)データ 読み込む
df = pd.read_csv('FEI_PREF_2015.csv').set_index('地域')
#データ中身確認
df.head().T

In [ ]:
#PCA
pca = PCA()
pca.fit(df[['情報通信業', '金融・保険業']])
#図示する
fig = plt.figure(figsize=(16, 7))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)
ax1.scatter(df['情報通信業'], df['金融・保険業'])
ax2.scatter(df['情報通信業'], df['金融・保険業'])
#第一主成分の軸を図示
y = (pca.components_[0][1] / pca.components_[0][0]) * (
	np.linspace(0, 0.15, 100) - np.mean(df['情報通信業'])) + np.mean(df['金融・保険業'])
ax2.plot(np.linspace(0, 0.15, 100), y, color='red')
#第二主成分の軸を図示
y = (pca.components_[1][1] / pca.components_[1][0]) * (
	np.linspace(0, 0.15, 100) - np.mean(df['情報通信業'])) + np.mean(df['金融・保険業'])
ax2.plot(np.linspace(0, 0.15, 100), y, color='red')
#中心を赤でplot
ax2.scatter(np.mean(df['情報通信業']), np.mean(df['金融・保険業']), color='red')
#目盛の範囲を調整
ax1.set_ylim(-0.05, 0.1)
ax2.set_ylim(-0.05, 0.1)

In [ ]:
#新しい直交座標系でプロット
fig = plt.figure(figsize=(14, 7))
ax = fig.add_subplot(1, 1, 1)
#元の系列
ax.scatter(
	pca.transform(df[['情報通信業', '金融・保険業']])[:, 0],
	pca.transform(df[['情報通信業', '金融・保険業']])[:, 1])
#第一主成分の座標
x = pca.transform(df[['情報通信業', '金融・保険業']])[:, 0]
y = x * 0
ax.scatter(x, y)
ax.plot(
	np.linspace(-0.02, 0.1, 100),
	np.linspace(-0.02, 0.02, 100) * 0,
	color='red')
#第二主成分の座標
y = pca.transform(df[['情報通信業', '金融・保険業']])[:, 1]
x = y * 0
ax.scatter(x, y)
ax.plot(
	np.linspace(-0.02, 0.1, 100) * 0,
	np.linspace(-0.02, 0.02, 100),
	color='red')

In [ ]:
#寄与度
print (pca.explained_variance_ratio_)

## 2-9-3 因子分析

In [ ]:
!pip install factor-analyzer

In [ ]:
#因子分析用のライブラリを読み込む
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt #可視化のため
import numpy.linalg as LA #固有値計算のため

#ESTAT 県民総生産 (2015)データ 読み込む
df = pd.read_csv('FEI_PREF_2015.csv').set_index('地域')
U = df

In [ ]:
#固有値の大きさ
ev, _ = LA.eig(U.corr())  #相関係数行列の固有値
plt.bar(np.arange(len(ev)), ev)  #固有値プロット
plt.plot([1] * 20, color='red')  #比較用のy=1の線をプロット

In [ ]:
#因子分析のインスタンス作成
fa = FactorAnalyzer(rotation=None, n_factors=5)  #回転させない
#モデリング
fa.fit(U)

In [ ]:
#因子負荷量 プロット
fig = plt.figure(figsize=(20,20))
#ax = []
for i in np.arange(5):
   ax = fig.add_subplot(3,2,i+1)
   plt.title('第'+str(i+1)+'因子負荷量')
   ax.bar([i[0:2] for i in U.columns],fa.loadings_[:,i]) #ラベルの先頭2文字のみ出力

## 2-9-4 多次元尺度構成法

In [ ]:
#多次元尺度構成法ライブラリ読み込む
from sklearn.manifold import MDS
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
#ワインデータ読み込み
data = load_wine()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
#MDSインスタンス作成しfitさせる
embedding = MDS(
	n_components=2, metric=True,
	random_state=1)  #metricのデフォルトはTrue 間隔尺度であるため計量MDSを実行
X_mds = embedding.fit_transform(X)

In [ ]:
#次元ごとのストレス値
l = []
for i in np.arange(len(X.columns)):
	embedding = MDS(
    	n_components=i + 1, metric=True,
    	random_state=1)  #metricのデフォルトはTrue 間隔尺度であるため計量MDSを実行
	X_mds = embedding.fit_transform(X)
	l.append(embedding.stress_)

plt.plot(np.arange(len(X.columns)) + 1, l, color='red')
plt.xlabel('次元数')
plt.ylabel('ストレス値')
plt.xlim(0, 13)
plt.xticks(np.arange(len(X.columns)) + 1)

In [ ]:
#2次元に配置する
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(1, 1, 1)
X_mds[y == 0, 0]
for i, j, k in zip(np.arange(3), ['blue', 'red', 'green'], ['^', 's', 'o']):
	ax.scatter(X_mds[y == i, 0], X_mds[y == i, 1], color=j, marker=k)

## 2-9-5 局所線形埋め込み

In [ ]:
#スパイラルデータ作成
x = np.array([[
	np.exp(-0.2 * (-2 * np.pi * (i / 360))) * np.cos(-2 * np.pi * (i / 360))
	for i in np.arange(0, 3000, 10)
], [
	np.exp(-0.2 * (-2 * np.pi * (i / 360))) * np.sin(-2 * np.pi * (i / 360))
	for i in np.arange(0, 3000, 10)
]]).T
#図示する
fig = plt.figure(figsize=(16, 7))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)
ax1.scatter(
	np.arange(0, 3000, 10),
	np.arange(0, 3000, 10) * 0,
	c=np.arange(0, 3000, 10),
	cmap=plt.cm.Spectral,
	s=5)
ax2.scatter(
	x[:, 0], x[:, 1], c=np.arange(0, 3000, 10), cmap=plt.cm.Spectral, s=20)
ax2.set_ylim(-40000, 30000)

In [ ]:
#主成分分析をしてみる
from sklearn.decomposition import PCA  #主成分分析
pca = PCA()
pca.fit(x)
#図示する
fig = plt.figure(figsize=(16, 7))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)
ax1.scatter(x[:, 0], x[:, 1], c=np.arange(0, 3000, 10), cmap=plt.cm.Spectral)
#第一主成分の軸を図示
y = (pca.components_[0][1] / pca.components_[0][0]) * (
	np.linspace(-10000, 10000, 100) - np.mean(x[:, 0])) + np.mean(x[:, 1])
ax1.plot(np.linspace(-5000, 5000, 100), y, color='red')
ax1.set_ylim(-40000, 30000)
#第一主成分を1次元で表す
ax2.scatter(
	pca.fit_transform(x)[:, 0],
	np.arange(0, 3000, 10) * 0,
	c=np.arange(0, 3000, 10),
	cmap=plt.cm.Spectral)

In [ ]:
#多様体学習ライブラリからLLEクラスをインポート
from sklearn.manifold import LocallyLinearEmbedding
#インスタンス化
embedding = LocallyLinearEmbedding(n_components=1,
                               	random_state=1)  #1次元に対応する
X_transformed = embedding.fit_transform(x)
#1次元のカラーマップ
fig = plt.figure(figsize=(16, 7))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(
	X_transformed,
	np.arange(0, 3000, 10) * 0,
	c=np.arange(0, 3000, 10),
	cmap=plt.cm.Spectral)

In [ ]:
#ワインデータ読み込み
data = load_wine()
embedding = LocallyLinearEmbedding(n_components=2, random_state=1)
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
X_transformed = embedding.fit_transform(X)
#2次元に配置する
fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(1, 1, 1)
for i, j, k in zip(np.arange(3), ['blue', 'red', 'green'], ['^', 's', 'o']):
	ax.scatter(
    	X_transformed[y == i, 0], X_transformed[y == i, 1], color=j, marker=k)

## 2-9-6 t-SNE

#### 2-9-6-2 t-SNE

In [ ]:
# t-SNE
from sklearn.manifold import TSNE
#ワインデータ読み込み
data = load_wine()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

In [ ]:
X_transformed = TSNE(
	n_components=2, perplexity=30, random_state=0).fit_transform(X)
#2次元に配置する
fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(1, 1, 1)
for i, j, k in zip(np.arange(3), ['blue', 'red', 'green'], ['^', 's', 'o']):
	ax.scatter(
    	X_transformed[y == i, 0], X_transformed[y == i, 1], color=j, marker=k)

# Chapter2-10 特徴量選択

※サポートサイトから2章用のデータをダウンロードし、「adult.data」をセッションストレージ（P.312）にアップして進めてください。

### 2-10-1 特徴量選択の3手法

In [ ]:
## ここで「You must restart the runtime in order to use newly installed versions.」というメッセージが出たら、「RESTART RUNTIME」ボタンをクリックしてランタイムを再起動してください。
!pip uninstall scikit-learn
!pip install scikit-learn==0.21.3

In [ ]:
!pip install category_encoders

In [ ]:
#データをトレーニング・テストに分けるモジュール
from sklearn.model_selection import train_test_split
#ロジスティック回帰モデル
from sklearn.linear_model import LogisticRegression
#正答率算出
from sklearn.metrics import accuracy_score
#category encoder インポート
import category_encoders as ce
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#データ 読み込み
df_adult = pd.read_csv('adult.data', header=None)
#変数名付与
df_adult.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum", "MaritalStatus",
    "Occupation", "Relationship", "Race", "Gender", "CapitalGain",
    "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

#ターゲット変数1/0エンコーディング
df_adult['Income'] = (df_adult['Income'] == ' >50K').astype(int)
#特徴量取得
X_adult = df_adult.drop(['Income'], axis=1)
#ターゲット変数を取得
y_adult = df_adult['Income']
#訓練・検証データを分離
X_adult_train, X_adult_test, y_adult_train, y_adult_test = train_test_split(
    X_adult, y_adult, test_size=0.2, random_state=1)

#ターゲットエンコーダ インスタンス化
encoder = ce.TargetEncoder(cols=['Education', 'Occupation', 'NativeCountry'])
enc = encoder.fit(X_adult_train, y_adult_train)
X_adult_train = enc.transform(X_adult_train)
X_adult_test = enc.transform(X_adult_test)

#One-Hotエンコーディング インスタンス化
encoder = ce.OneHotEncoder(
    cols=['WorkClass', 'MaritalStatus', 'Relationship', 'Race', 'Gender'],
    use_cat_names=True)
enc = encoder.fit(X_adult_train)
X_adult_train = enc.transform(X_adult_train)
X_adult_test = enc.transform(X_adult_test)

#One-Hotエンコーディングで冗長になった変数を落とす
drop_l = [
    [col for col in X_adult_train.columns if col.find('WorkClass') >= 0][0], [
        col for col in X_adult_train.columns if col.find('MaritalStatus') >= 0
    ][0], [
        col for col in X_adult_train.columns if col.find('Relationship') >= 0
    ][0], [col for col in X_adult_train.columns if col.find('Race') >= 0][0],
    [col for col in X_adult_train.columns if col.find('Gender') >= 0][0]
]
X_adult_train = X_adult_train.drop(drop_l, axis=1)
X_adult_test = X_adult_test.drop(drop_l, axis=1)

In [ ]:
#すべての特徴量
features_all = X_adult_train.columns
print('特徴量 : {} '.format(features_all))
#比較用として全ての特徴量でモデル作成
#フルモデルと比較
reg = LogisticRegression().fit(X_adult_train, y_adult_train)
score_all = accuracy_score(y_adult_test, reg.predict(X_adult_test))
print('訓練 : {} '.format(
    accuracy_score(y_adult_train, reg.predict(X_adult_train))))
print('検証 : {} '.format(
    accuracy_score(y_adult_test, reg.predict(X_adult_test))))

## 2-10-2 Filter法

In [ ]:
#単変量Filterモジュール読み込む
from sklearn.feature_selection import SelectKBest, f_regression, f_classif, chi2
#F値でfeature selection filter法
tr_score = []  #訓練スコアを入れるリスト
te_score = []  #検証スコアを入れるリスト
features = []  #特徴量を入れるりスト
#Kごとに精度評価
for i in np.arange(len(X_adult_train.columns)):
    sel = SelectKBest(f_classif, k=i + 1).fit(X_adult_train, y_adult_train)
    X_train_sel = sel.transform(X_adult_train)
    X_test_sel = sel.transform(X_adult_test)
    reg = LogisticRegression().fit(X_train_sel, y_adult_train)
    tr_score.append(accuracy_score(y_adult_train, reg.predict(X_train_sel)))
    te_score.append(accuracy_score(y_adult_test, reg.predict(X_test_sel)))
    features.append(X_adult_train.columns[sel.get_support()])

#kごとに精度がどのように変わったか可視化する
plt.figure(figsize=(14, 7))
plt.plot(np.arange(len(X_adult_train.columns)) + 1, tr_score, label='Train')
plt.plot(np.arange(len(X_adult_train.columns)) + 1, te_score, label='Test')
plt.ylabel('Accuracy')
plt.xlabel('The Number of features')
plt.xticks(np.arange(1, 1 + len(X_adult_train.columns)))
plt.legend(fontsize=14)
plt.grid()
plt.title('Feature selection by using F value')

In [ ]:
#訓練データの精度最大となるk
print (tr_score.index(np.max(tr_score)))

In [ ]:
#訓練データで精度最大になる変数の数と組み合わせ
features_sel_filter = features[tr_score.index(np.max(tr_score))]
print ('特徴量 : {} '.format(features_sel_filter))
print ('訓練 : {} '.format((tr_score[tr_score.index(np.max(tr_score))])))
print ('検証 : {} '.format((te_score[tr_score.index(np.max(tr_score))])))

## 2-10-3 Wrapper法

In [ ]:
!pip install -U mlxtend

In [ ]:
#wrapper法実行モジュール読み込む
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

####2-10-3-1 Sequence Forward Generation

In [ ]:
#SFSインスタンス化
sfs = SFS(LogisticRegression(),  #利用するモデル
           k_features=(1,33), #特徴量の数 1-33個の間でCVスコアベストを探す
           forward=True, #フォワード型
           floating=False, #フローティングの有無
           verbose=2,
           scoring='accuracy',#精度指標
           cv=5,
          n_jobs=-1)

sfs = sfs.fit(X_adult_train, y_adult_train)

In [ ]:
fig1 = plot_sfs(sfs.get_metric_dict(), kind='std_dev', figsize=(14, 7))
plt.title('Sequence Forward Generation (w. StdDev)')
plt.grid()

In [ ]:
#選ばれた特徴量のサブセットでモデルを作成
features_sel_wrapper_sfg = list(sfs.k_feature_names_)
reg = LogisticRegression().fit(X_adult_train[features_sel_wrapper_sfg],
                               y_adult_train)
print('特徴量 : {} '.format(features_sel_wrapper_sfg))
print('訓練 : {} '.format(
    accuracy_score(y_adult_train,
                   reg.predict(X_adult_train[features_sel_wrapper_sfg]))))
print('検証 : {} '.format(
    accuracy_score(y_adult_test,
                   reg.predict(X_adult_test[features_sel_wrapper_sfg]))))
score_sel_wrapper_sfg = accuracy_score(
    y_adult_test, reg.predict(X_adult_test[features_sel_wrapper_sfg]))

#### 2-10-3-2 Sequential Backward Generation

In [ ]:
#SFSインスタンス化
sbs = SFS(LogisticRegression(),  #利用するモデル
           k_features=(1,33), #特徴量の数 1-33個の間でCVスコアベストを探す
           forward=False, #バックワード型
           floating=False, #フローティングの有無
           verbose=2,
           scoring='accuracy',#精度指標
           cv=5,
          n_jobs=-1)

sbs = sbs.fit(X_adult_train, y_adult_train)

In [ ]:
fig1 = plot_sfs(sbs.get_metric_dict(), kind='std_dev', figsize=(14, 7))
plt.title('Sequence Backward Generation (w. StdDev)')
plt.grid()

In [ ]:
#選ばれた特徴量のサブセットでモデルを作成
features_sel_wrapper_sbg = list(sbs.k_feature_names_)
reg = LogisticRegression().fit(X_adult_train[features_sel_wrapper_sbg],
                               y_adult_train)
print('特徴量 : {} '.format(features_sel_wrapper_sbg))
print('訓練 : {} '.format(
    accuracy_score(y_adult_train,
                   reg.predict(X_adult_train[features_sel_wrapper_sbg]))))
print('検証 : {} '.format(
    accuracy_score(y_adult_test,
                   reg.predict(X_adult_test[features_sel_wrapper_sbg]))))
score_sel_wrapper_sbg = accuracy_score(
    y_adult_test, reg.predict(X_adult_test[features_sel_wrapper_sbg]))

####2-10-3-3 Bidirectional Generation

In [ ]:
#SFSインスタンス化
sfsf = SFS(
    LogisticRegression(),  #利用するモデル
    k_features=(1, 33),  #特徴量の数 1-33個の間でCVスコアベストを探す
    forward=True,  #フォワード型
    floating=True,  #フローティングの有無 Trueとする
    verbose=2,
    scoring='accuracy',  #精度指標
    cv=5,
    n_jobs=-1)

sfsf = sfsf.fit(X_adult_train, y_adult_train)

In [ ]:
fig1 = plot_sfs(sfsf.get_metric_dict(), kind='std_dev', figsize=(14, 7))
plt.title('Bidirectional Generation (w. StdDev)')
plt.grid()

In [ ]:
#選ばれた特徴量のサブセットでモデルを作成
features_sel_wrapper_bg = list(sfsf.k_feature_names_)
reg = LogisticRegression().fit(X_adult_train[features_sel_wrapper_bg],
                               y_adult_train)
print('特徴量 : {} '.format(features_sel_wrapper_bg))
print('訓練 : {} '.format(
    accuracy_score(y_adult_train,
                   reg.predict(X_adult_train[features_sel_wrapper_bg]))))
print('検証 : {} '.format(
    accuracy_score(y_adult_test,
                   reg.predict(X_adult_test[features_sel_wrapper_bg]))))
score_sel_wrapper_bg = accuracy_score(
    y_adult_test, reg.predict(X_adult_test[features_sel_wrapper_bg]))

# 2-10-4 Embedded法

In [ ]:
#SelectFromModelモジュール読み込み
from sklearn.feature_selection import SelectFromModel
#標準化モジュール読み込む
from sklearn.preprocessing import StandardScaler

####2-10-4-1 Lasso回帰

In [ ]:
#標準化
scaler = StandardScaler()
scaler.fit(X_adult_train)
X_adult_train_standard = scaler.transform(X_adult_train)
X_adult_test_standard = scaler.transform(X_adult_test)

In [ ]:
#lasso selectFromModelインスタンス作成
embeded_selector = SelectFromModel(
    LogisticRegression(C=1.0, penalty="l1", solver='liblinear'),
    threshold='mean')
embeded_selector.fit(X_adult_train_standard, y_adult_train)

In [ ]:
#選ばれた特徴量のサブセットでモデルを作成
features_sel_embedded_lasso = X_adult_train.columns[
    embeded_selector.get_support()]
reg = LogisticRegression()
#モデル作成
reg.fit(embeded_selector.transform(X_adult_train), y_adult_train)
print('特徴量 : {} '.format(features_sel_embedded_lasso))
print('訓練 : {} '.format(
    accuracy_score(y_adult_train,
                   reg.predict(embeded_selector.transform(X_adult_train)))))
print('検証 : {} '.format(
    accuracy_score(y_adult_test,
                   reg.predict(embeded_selector.transform(X_adult_test)))))
score_sel_embedded_lasso = accuracy_score(
    y_adult_test, reg.predict(embeded_selector.transform(X_adult_test)))

#####2-10-4-2 RandomForest

In [ ]:
#RaondomForest
from sklearn.ensemble import RandomForestClassifier

#RondomForest SelectFromModelインスタンス作成
embeded_selector = SelectFromModel(
    RandomForestClassifier(
        n_estimators=100, random_state=0, min_samples_leaf=50), "mean")
embeded_selector.fit(X_adult_train, y_adult_train)

#選ばれた特徴量のサブセットでモデルを作成
features_sel_embedded_rf = X_adult_train.columns[
    embeded_selector.get_support()]
reg = LogisticRegression()
#モデル作成
reg.fit(embeded_selector.transform(X_adult_train), y_adult_train)
print('特徴量 : {} '.format(features_sel_embedded_rf))
print('訓練 : {} '.format(
    accuracy_score(y_adult_train,
                   reg.predict(embeded_selector.transform(X_adult_train)))))
print('検証 : {} '.format(
    accuracy_score(y_adult_test,
                   reg.predict(embeded_selector.transform(X_adult_test)))))
score_sel_embedded_rf = accuracy_score(
    y_adult_test, reg.predict(embeded_selector.transform(X_adult_test)))